In [1]:
import image_tools
import cv2
import numpy as np
import os

# 상수 정의
IMAGE_FILE = "bird.png"
WINDOW_IMAGE = "Image"

# 글로벌 변수
drawing = False  # 드래그 중인지 여부
start_x, start_y = -1, -1  # 시작 좌표
current_shape = None  # 현재 선택된 도형
original_img = None  # 원본 이미지 저장

def draw_star(img: np.ndarray, start: tuple, end: tuple, color: tuple = (0, 255, 0), thickness: int = 2) -> None:
    """별 모양을 그립니다."""
    x1, y1 = start
    x2, y2 = end
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    radius = max(abs(x2 - x1), abs(y2 - y1)) // 2
    outer_points = []
    inner_points = []
    for i in range(10):
        angle = i * np.pi / 5
        r = radius if i % 2 == 0 else radius // 2
        x = int(center[0] + r * np.cos(angle))
        y = int(center[1] + r * np.sin(angle))
        (outer_points if i % 2 == 0 else inner_points).append((x, y))
    star_points = []
    for i in range(5):
        star_points.append(outer_points[i])
        star_points.append(inner_points[i])
    star_points = np.array(star_points, np.int32).reshape((-1, 1, 2))
    cv2.polylines(img, [star_points], True, color, thickness)

def draw_polygon_shape(img: np.ndarray, start: tuple, end: tuple, sides: int, color: tuple = (0, 255, 0), thickness: int = 2) -> None:
    """다각형을 그립니다."""
    x1, y1 = start
    x2, y2 = end
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    radius = max(abs(x2 - x1), abs(y2 - y1)) // 2
    points = []
    for i in range(sides):
        angle = 2 * np.pi * i / sides - np.pi / 2
        x = int(center[0] + radius * np.cos(angle))
        y = int(center[1] + radius * np.sin(angle))
        points.append((x, y))
    points = np.array(points, np.int32).reshape((-1, 1, 2))
    cv2.polylines(img, [points], True, color, thickness)

def draw_rectangle(img: np.ndarray, start: tuple, end: tuple, color: tuple = (0, 255, 0), thickness: int = 2) -> None:
    """사각형을 그립니다."""
    x1, y1 = start
    x2, y2 = end
    x1, x2 = min(x1, x2), max(x1, x2)
    y1, y2 = min(y1, y2), max(y1, y2)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)

def draw_polygon(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    """마우스 이벤트를 처리하여 다각형을 그립니다."""
    global drawing, start_x, start_y, img, current_shape

    if event == cv2.EVENT_LBUTTONDOWN:
        if current_shape:
            drawing = True
            start_x, start_y = x, y
            img = original_img.copy()  # 원본 이미지로 초기화

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing and current_shape:
            img_copy = original_img.copy()
            if current_shape == 'star':
                draw_star(img_copy, (start_x, start_y), (x, y))
            elif current_shape == 'triangle':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 3)
            elif current_shape == 'rectangle':
                draw_rectangle(img_copy, (start_x, start_y), (x, y))
            elif current_shape == 'pentagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 5)
            elif current_shape == 'hexagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 6)
            elif current_shape == 'octagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 8)
            elif current_shape == 'decagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 10)
            cv2.imshow(WINDOW_IMAGE, img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        if drawing and current_shape:
            drawing = False
            img_copy = original_img.copy()
            if current_shape == 'star':
                draw_star(img_copy, (start_x, start_y), (x, y))
            elif current_shape == 'triangle':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 3)
            elif current_shape == 'rectangle':
                draw_rectangle(img_copy, (start_x, start_y), (x, y))
            elif current_shape == 'pentagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 5)
            elif current_shape == 'hexagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 6)
            elif current_shape == 'octagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 8)
            elif current_shape == 'decagon':
                draw_polygon_shape(img_copy, (start_x, start_y), (x, y), 10)
            img[:] = img_copy[:]
            cv2.imshow(WINDOW_IMAGE, img)

try:
    # 이미지 로드
    img = image_tools.load_image(IMAGE_FILE).copy()
    original_img = img.copy()  # 원본 이미지 저장

    # 윈도우 생성
    cv2.namedWindow(WINDOW_IMAGE, cv2.WINDOW_AUTOSIZE)

    # 마우스 콜백 설정
    cv2.setMouseCallback(WINDOW_IMAGE, draw_polygon)

    # 초기 이미지 표시
    cv2.imshow(WINDOW_IMAGE, img)

    # 키 입력 처리
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == ord('Q'):
            break
        elif key == ord('1'):
            current_shape = 'star'
        elif key == ord('2'):
            current_shape = 'triangle'
        elif key == ord('3'):
            current_shape = 'rectangle'
        elif key == ord('4'):
            current_shape = 'octagon'
        elif key == ord('5'):
            current_shape = 'pentagon'
        elif key == ord('6'):
            current_shape = 'decagon'
        elif key == ord('7'):
            current_shape = 'hexagon'

except (FileNotFoundError, ValueError) as e:
    print(f"오류: {e}")
finally:
    cv2.destroyAllWindows()

KeyboardInterrupt: 